# 基板の大きさの変化によるスパッタリング率の変化を見る

[参照]
https://www.notion.so/048e4f972fb44a81a6a4f65e8339e228?pvs=4

In [1]:
import os
import shutil
from pydantic import BaseModel

In [2]:
path2target = "/Users/y1u0d2/desktop/Lab/result/lammps/Si/sputtering/tersoff/analyze_sp_with_size_of_substrate"
path2template = os.path.join(path2target, "template")
path2result = os.path.join(path2target, "result")


INPUT_FILE = "in.tersoff"
SHELL_SCRIPT = "in.sh"
POTENTIAL_FILE = "SiC.tersoff.zbl"
POTENTIAL_NAME = "TERSOFF"

In [3]:
width_list = [8.5]
depth_list = [15, 20]
energy_list = [200, 500, 750, 1000]

width_num_injection_map = {
    7.5: 1200,
    8.5: 1600,
    9.5: 2000
}

# 各スパッタリングシミュレーションのConfig作成
class SubstrateSizeAnalysisSputteringConfig(BaseModel):
    potential_name: str
    width: float
    depth: float
    height: float
    energy: int
    num_injection: int


def get_height(depth: float) -> float:
    return depth + 2.5

In [4]:
# width_list, depth_list, energy_listの組み合わせでConfigを作成
config_list = []
for width in width_list:
    for depth in depth_list:
        for energy in energy_list:
            height = get_height(depth)
            num_injection = width_num_injection_map[width]
            config = SubstrateSizeAnalysisSputteringConfig(
                potential_name=POTENTIAL_NAME,
                width=width,
                depth=depth,
                height=height,
                energy=energy,
                num_injection=num_injection
            )
            config_list.append(config)

In [5]:
# 各Configに対応するディレクトリを作成し、入力ファイル、シェルスクリプト、ポテンシャルファイルをコピー
from typing import List
def change_input_parameter(lines: List[str], keyword: str, new_value: int):
    for idx, line in enumerate(lines):
        if keyword in line:
            lines[idx] = f"{keyword} {new_value}"
            return lines
    raise ValueError(f"keyword: {keyword} is not found.")

for config in config_list:
    # ディレクトリを作成
    dir_name = f"{config.potential_name}_W-{config.width}_D-{config.depth}_E-{config.energy}"
    path2dir = os.path.join(path2result, dir_name)
    os.makedirs(path2dir, exist_ok=True)

    # width, depth, height, energy, num_injectionを入力ファイルに書き込む
    with open(os.path.join(path2template, INPUT_FILE), mode="r") as f:
        lines = [s.strip() for s in f.readlines()]
    
    lines = change_input_parameter(
        lines=lines,
        keyword="variable lenx index",
        new_value=config.width
    )
    lines = change_input_parameter(
        lines=lines,
        keyword="variable leny index",
        new_value=config.width
    )
    lines = change_input_parameter(
        lines=lines,
        keyword="variable hgt  index",
        new_value=config.depth
    )
    lines = change_input_parameter(
        lines=lines,
        keyword="variable hgtB index",
        new_value=config.height
    )
    lines = change_input_parameter(
        lines=lines,
        keyword="variable lp equal",
        new_value=config.num_injection
    )
    lines = change_input_parameter(
        lines=lines,
        keyword="variable enrg equal",
        new_value=config.energy
    )

    # linesをファイルに書き込む
    with open(os.path.join(path2dir, INPUT_FILE), mode="w") as f:
        f.write("\n".join(lines))
    
    # シェルスクリプトをコピー
    shutil.copy(os.path.join(path2template, SHELL_SCRIPT), os.path.join(path2dir, SHELL_SCRIPT))
    shutil.copy(os.path.join(path2template, POTENTIAL_FILE), os.path.join(path2dir, POTENTIAL_FILE))